# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,flying fish cove,-10.4217,105.6791,25.82,88,75,4.12,CX,1698269479
1,1,zhangatas,43.5667,69.7500,16.17,47,100,8.17,KZ,1698269644
2,2,chernyshevskiy,63.0128,112.4714,-7.51,97,100,1.73,RU,1698269645
3,3,jamestown,42.0970,-79.2353,21.68,35,0,4.12,US,1698269395
4,4,lebu,-37.6167,-73.6500,11.92,79,0,10.87,CL,1698269647


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
%%capture --no-display

# Configure the map plot
map_plot_1=city_data_df.hvplot.points("Lng",
     "Lat",  geo = True, tiles = "OSM", frame_width = 800, frame_height = 600, size = "Humidity", 
                                      scale=1.1, color="City", alpha=0.5)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities=city_data_df.loc[(city_data_df['Max Temp']<=30) & (city_data_df['Max Temp']>=20)& (city_data_df['Wind Speed']<=4)
                             & (city_data_df['Cloudiness']==0)].dropna()


# Display sample data
ideal_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,crane,31.3974,-102.3501,25.06,41,0,2.63,US,1698269663
110,110,toccoa,34.5773,-83.3324,24.36,53,0,2.57,US,1698269744
122,122,saint-pierre,-21.3393,55.4781,24.20,73,0,2.06,RE,1698269756
137,137,darwin,-12.4611,130.8418,26.99,85,0,0.51,AU,1698269768
143,143,tamanrasset,22.7850,5.5228,22.95,16,0,1.54,DZ,1698269773


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_cities.loc[:,['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""
# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
20,crane,US,31.3974,-102.3501,41,
110,toccoa,US,34.5773,-83.3324,53,
122,saint-pierre,RE,-21.3393,55.4781,73,
137,darwin,AU,-12.4611,130.8418,85,
143,tamanrasset,DZ,22.7850,5.5228,16,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
categories='accommodation.hotel'
apiKey=geoapify_key
params = {"categories":categories,
          'apiKey':geoapify_key,

}

# Print a message to follow up the hotel search
print("Starting hotel search")

for index, row in hotel_df.iterrows():
    lat=row['Lat']
    long=row['Lng']
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_addressA = name_address.json()
#     print(name_addressA)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_addressA["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
crane - nearest hotel: No hotel found
toccoa - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
darwin - nearest hotel: Mantra Pantanas Darwin
tamanrasset - nearest hotel: فندق الأمان
maun - nearest hotel: Center Lodge Conference Center
emerald - nearest hotel: Emerald Central Hotel
nishinoomote - nearest hotel: ゼウスハウス
sterling - nearest hotel: Residence Inn by Marriott Dulles Airport at Dulles 28 Centre
piti village - nearest hotel: The Cliff Hotel
cutrofiano - nearest hotel: Villa Padula
kotputli - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found
bahia de kino - nearest hotel: Hotel Hacienda
point pleasant - nearest hotel: Crystal Point Inn
fredericksburg - nearest hotel: Kenmore Inn
biltine - nearest hotel: فندق ثلاث نجوم بلتن
reguiba - nearest hotel: No hotel found
munuf - nearest hotel: No hotel found
mersa matruh - nearest hotel: صقور
chios - nearest hotel: Aegean Sea


,City,Country,Lat,Lng,Humidity,Hotel Name
20,crane,US,31.3974,-102.3501,41,No hotel found
110,toccoa,US,34.5773,-83.3324,53,No hotel found
122,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
137,darwin,AU,-12.4611,130.8418,85,Mantra Pantanas Darwin
143,tamanrasset,DZ,22.7850,5.5228,16,فندق الأمان
210,maun,BW,-19.9833,23.4167,13,Center Lodge Conference Center
224,emerald,AU,-23.5333,148.1667,61,Emerald Central Hotel
247,nishinoomote,JP,30.7333,131.0000,59,ゼウスハウス
249,sterling,US,39.0062,-77.4286,52,Residence Inn by Marriott Dulles Airport at Du...
275,piti village,GU,13.4626,144.6933,87,The Cliff Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot_2=hotel_df.hvplot.points(
"Lng",
"Lat",
geo=True,
tiles="OSM",
frame_width=800,
frame_height=600,
alpha=0.5,
size="Humidity",
color="City",
scale=1.1,
hover_cols=['Hotel Name', 'Country'])
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)